# Introduction

# Create database

In [1]:
# Creates a small SQLite database (shop.db) with:
# - customers
# - products
# - orders

import sqlite3

# Connect to database (creates file if it does not exist)
conn = sqlite3.connect("shop.db")
cur = conn.cursor()

# Enable foreign keys in SQLite
# “If I delete a customer, all their orders are deleted automatically.”
# ⚠️ But without PRAGMA foreign_keys = ON;, this does NOT happen in SQLite!
cur.execute("PRAGMA foreign_keys = ON;")

# --------------------------------------------------
# Drop tables (so script can be run multiple times)
# --------------------------------------------------
cur.execute("DROP TABLE IF EXISTS orders;")
cur.execute("DROP TABLE IF EXISTS customers;")
cur.execute("DROP TABLE IF EXISTS products;")

# --------------------------------------------------
# Create tables
# --------------------------------------------------

# each customer will get a primary key:
# - a column that uniquely identifies each row
# - no duplicates allowed
# - cannot be NULL
# With AUTOINCREMENT:
# - SQLite automatically generates the next number
# - You do not have to provide the ID yourself
cur.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT
);
""")

cur.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price REAL NOT NULL
);
""")

# here we use FOREIGN KEY references
# A foreign key is a column that points to the primary key of another table.
# These lines say:
#    “Each order must belong to a real customer and a real product.”
#    “If a customer is deleted, delete all their orders automatically.”
# Without foreign keys
# - Orders could reference customers that don’t exist!
# - Orders could reference products that don’t exist!

cur.execute("""
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(id) ON DELETE CASCADE,
    FOREIGN KEY (product_id) REFERENCES products(id)
);
""")

# --------------------------------------------------
# Insert data using lists + executemany()
# --------------------------------------------------

# Customers: (name, email)
customers = [
    ("Alice", "alice@example.com"),
    ("Bob",   "bob@example.com"),
    ("Carla", "carla@example.com"),
    ("David", "david@example.com"),
]

cur.executemany(
    "INSERT INTO customers (name, email) VALUES (?, ?);",
    customers
)

# Products: (name, price)
products = [
    ("Notebook", 2.50),
    ("Pen", 1.20),
    ("Backpack", 29.99),
    ("Water Bottle", 9.90),
    ("USB Stick", 7.50),
]

cur.executemany(
    "INSERT INTO products (name, price) VALUES (?, ?);",
    products
)

# Orders:
# (customer_id, product_id, quantity, order_date)
orders = [
    # Alice (id=1) orders 4 different products
    (1, 1, 1, "2026-01-10"),
    (1, 2, 2, "2026-01-10"),
    (1, 3, 1, "2026-01-10"),
    (1, 4, 1, "2026-01-10"),

    # Bob (id=2) orders 2 products
    (2, 2, 5, "2026-01-11"),
    (2, 5, 1, "2026-01-11"),

    # Carla (id=3) orders 4 products
    (3, 1, 2, "2026-01-12"),
    (3, 2, 1, "2026-01-12"),
    (3, 4, 3, "2026-01-12"),
    (3, 5, 1, "2026-01-12"),

    # David (id=4) orders 1 product
    (4, 3, 1, "2026-01-13"),
]

cur.executemany(
    """
    INSERT INTO orders (customer_id, product_id, quantity, order_date)
    VALUES (?, ?, ?, ?);
    """,
    orders
)

# --------------------------------------------------
# Save and close
# --------------------------------------------------
conn.commit()
conn.close()

print("Database shop.db created with sample data.")

Database shop.db created with sample data.


# Retrieve data: Customer names

In [2]:
# Opens shop.db and prints all customer names.

import sqlite3

# Connect to the database
conn = sqlite3.connect("shop.db")
cur = conn.cursor()

# Select only customer names
cur.execute("SELECT name FROM customers;")

rows = cur.fetchall()

print("Customers:")
for row in rows:
    # row is a tuple, even with one column
    print("-", row[0])

# Close the database connection
conn.close()

Customers:
- Alice
- Bob
- Carla
- David


# Retrieve data: What did Bob order?

In [3]:
# Print all product names and quantities that Bob ordered.
# Variant 1

import sqlite3

# Connect to database
conn = sqlite3.connect("shop.db")
cur = conn.cursor()

# SQL query with JOINs
cur.execute("""
SELECT products.name, orders.quantity
FROM orders
JOIN customers ON orders.customer_id = customers.id
JOIN products  ON orders.product_id  = products.id
WHERE customers.name = 'Bob';
""")

rows = cur.fetchall()

print("Products ordered by Bob:")

if not rows:
    print("Bob has no orders.")
else:
    for row in rows:
        product_name = row[0]
        quantity = row[1]
        print("-", product_name, ":", quantity)

# Close connection
conn.close()

Products ordered by Bob:
- Pen : 5
- USB Stick : 1


In [4]:
# Print all product names and quantities that Bob ordered.
# Variant 2: NO JOIN version – step by step

import sqlite3

# Connect to database
conn = sqlite3.connect("shop.db")
cur = conn.cursor()

# 1) Get Bob's customer id
cur.execute(
    "SELECT id FROM customers WHERE name = ?;",
    ("Bob",)
)
row = cur.fetchone()

if row is None:
    print("Bob not found.")
    conn.close()
    exit()

bob_id = row[0]


# 2) Get Bob's orders
cur.execute(
    "SELECT product_id, quantity FROM orders WHERE customer_id = ?;",
    (bob_id,)
)
orders = cur.fetchall()

print("Products ordered by Bob:")
if not orders:
    print("Bob has no orders.")
else:
    # 3) For each order, get product name    
    for order in orders:
        product_id = order[0]
        quantity = order[1]

        cur.execute(
            "SELECT name FROM products WHERE id = ?;",
            (product_id,)
        )
        product_row = cur.fetchone()
        product_name = product_row[0]
        print("-", product_name, ":", quantity)

# Close connection
conn.close()

Products ordered by Bob:
- Pen : 5
- USB Stick : 1


# What is a join?

In [5]:
# Pandas join example:
# Show product names and quantities that Bob ordered.

import pandas as pd

# 1) Create example tables
customers = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Carla"]
})

products = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ["Notebook", "Pen", "USB Stick"]
})

orders = pd.DataFrame({
    "customer_id": [2, 2, 3],
    "product_id": [2, 3, 1],
    "quantity": [5, 1, 2]
})

In [6]:
customers

,id,name
0,1,Alice
1,2,Bob
2,3,Carla


In [7]:
products

,id,name
0,1,Notebook
1,2,Pen
2,3,USB Stick


In [8]:
orders

,customer_id,product_id,quantity
0,2,2,5
1,2,3,1
2,3,1,2


In [9]:
# 2) Join orders with customers
orders_with_customers = pd.merge(
    orders,
    customers,
    left_on="customer_id",
    right_on="id"
)
orders_with_customers

,customer_id,product_id,quantity,id,name
0,2,2,5,2,Bob
1,2,3,1,2,Bob
2,3,1,2,3,Carla


In [10]:
# 3) Join result with products
full_table = pd.merge(
    orders_with_customers,
    products,
    left_on="product_id",
    right_on="id",
    suffixes=("_customer", "_product")
)
full_table

,customer_id,product_id,quantity,id_customer,name_customer,id_product,name_product
0,2,2,5,2,Bob,2,Pen
1,2,3,1,2,Bob,3,USB Stick
2,3,1,2,3,Carla,1,Notebook


In [11]:
# 4) Filter only Bob
bob_orders = full_table[full_table["name_customer"] == "Bob"]

# Show only relevant columns
result = bob_orders[["name_product", "quantity"]]
result

,name_product,quantity
0,Pen,5
1,USB Stick,1


# Difference: Inner vs. Left join

Note:
- INNER JOIN keeps only rows that match in both tables.
- LEFT JOIN keeps all rows from the left table, even if there is no match.
- RIGHT JOIN exists in principle as well, but you almost never need it — just swap the tables and use LEFT JOIN
- OUTER JOIN keeps rows even when there is no match.

In [12]:
import pandas as pd

# Left table
customers = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Carla"]
})

# Right table (note: Carla has no order)
orders = pd.DataFrame({
    "customer_id": [1, 2],
    "product": ["Notebook", "Pen"]
})

In [13]:
customers

,id,name
0,1,Alice
1,2,Bob
2,3,Carla


In [14]:
orders

,customer_id,product
0,1,Notebook
1,2,Pen


In [15]:
# INNER JOIN
# Carla will be missing because she has no order.
inner = pd.merge(
    customers,
    orders,
    left_on="id",
    right_on="customer_id",
    how="inner"
)
inner

,id,name,customer_id,product
0,1,Alice,1,Notebook
1,2,Bob,2,Pen


In [16]:
# LEFT JOIN
left = pd.merge(
    customers,
    orders,
    left_on="id",
    right_on="customer_id",
    how="left"
)
left

,id,name,customer_id,product
0,1,Alice,1.0,Notebook
1,2,Bob,2.0,Pen
2,3,Carla,NaN,NaN


# Change data

In [17]:
# Opens shop.db and:
# - deletes a customer (example: Bob)
# - inserts a new customer (Erin)
# - increases price of a product by 10% (example: USB Stick)

import sqlite3

conn = sqlite3.connect("shop.db")
cur = conn.cursor()

# Enable foreign keys (so ON DELETE CASCADE works)
cur.execute("PRAGMA foreign_keys = ON;")

# -------------------------
# 1) DELETE a customer
# -------------------------
name_to_delete = "Bob"

# Find the customer id
cur.execute("SELECT id FROM customers WHERE name = ?;", (name_to_delete,))
row = cur.fetchone()

if row is None:
    print("Customer not found, nothing deleted:", name_to_delete)
else:
    customer_id = row[0]
    cur.execute("DELETE FROM customers WHERE id = ?;", (customer_id,))
    print("Deleted customer:", name_to_delete, "(id:", customer_id, ")")

# -------------------------
# 2) INSERT a new customer
# -------------------------
new_name = "Erin"
new_email = "erin@example.com"

cur.execute("INSERT INTO customers (name, email) VALUES (?, ?);", (new_name, new_email))
new_id = cur.lastrowid
print("Inserted new customer:", new_name, "(id:", new_id, ")")

# -------------------------
# 3) INCREASE product price by 10%
# -------------------------
product_name = "USB Stick"

# Read old price
cur.execute("SELECT id, price FROM products WHERE name = ?;", (product_name,))
row = cur.fetchone()

if row is None:
    print("Product not found, no price update:", product_name)
else:
    product_id = row[0]
    old_price = row[1]

    new_price = old_price * 1.10   # 10% increase
    new_price = round(new_price, 2)

    cur.execute("UPDATE products SET price = ? WHERE id = ?;", (new_price, product_id))
    print("Updated price for:", product_name)
    print("Old price:", old_price)
    print("New price:", new_price)

# Save changes and close
conn.commit()
conn.close()

print("Done! Changes saved.")

Deleted customer: Bob (id: 2 )
Inserted new customer: Erin (id: 5 )
Updated price for: USB Stick
Old price: 7.5
New price: 8.25
Done! Changes saved.


# SQL DB and Pandas

In [18]:
import pandas as pd
import sqlite3

# 1) Create a small DataFrame in Pandas
df_customers = pd.DataFrame({
    "name": ["Alice", "Bob", "Carla"],
    "city": ["Berlin", "Munich", "Berlin"]
})
df_customers

,name,city
0,Alice,Berlin
1,Bob,Munich
2,Carla,Berlin


In [19]:
# 2) Create / open SQLite database
conn = sqlite3.connect("mini_shop.db")

# 3) Store DataFrame into SQLite table
# to_sql returns the number of rows affected by to_sql()
df_customers.to_sql("customers", conn, if_exists="replace", index=False)

3

In [20]:
# 4) Retrieve information from SQLite as a DataFrame
df_from_db = pd.read_sql_query("SELECT * FROM customers;", conn)
df_from_db

,name,city
0,Alice,Berlin
1,Bob,Munich
2,Carla,Berlin


In [21]:
df_city_count = pd.read_sql_query("""
SELECT city, COUNT(*) AS customer_count
FROM customers
GROUP BY city;
""", conn)

df_city_count

,city,customer_count
0,Berlin,2
1,Munich,1


In [22]:
conn.close()